In [ ]:
import numpy as np
import random
import time
import math
import itertools


#функция для чтения из файла
def readFile(name):
    file=open(name)
    text = file.read()
    file.close()
    return text

#функция для запись в файл вектора
def writeFile(text, name):
    file=open(name, 'w')
    file.write(text)
    file.close()


def text_to_vector(text):
  res =[]
  for x in text:
    res.append(ord(x))
  return res
  
# xor для 2-х строк
def new_xor(m, c):
  while len(c)<len(m):
    c+=' '
  res =[]
  if len(m)==len(c):
    for (m_bit, c_bit) in zip(m, c):
      res.append(chr(ord(m_bit)^ord(c_bit)))
    #print(res)
  return ''.join(str(x) for x in res)




# матрица в строку
def makeMatrixFromString(strin, column, row):
    if len(strin)<column*row:
        #print(len(strin))
        #print(column*row)
        extra=column*row-len(strin)
        for i in range(0, extra):
            strin=strin+"*"
    matrix_strings = [strin[i:i+column] for i in range(0, len(strin), column)]  
    #print(matrix_strings)
    i=0
    matrix_symbs=[[0 for j in range(column)] for i in range(row)]
    for strin in matrix_strings:
        symbs = [strin[i:i+1] for i in range(0, len(strin), 1)]
        matrix_symbs[i] = symbs
        i+=1
    #print(matrix_symbs)
    return matrix_symbs


def encrypt_Skitala(str, column, row):
    matrix_symbs = makeMatrixFromString(str, column, row)
    matrix=np.array(matrix_symbs)
    encryptMarix=matrix.transpose()
    encryptedStr=''
    for i in range ( len(encryptMarix) ): 
          for j in range ( len(encryptMarix[i]) ):
                encryptedStr+=encryptMarix[i][j]
    return encryptedStr

#функция для расшифровки Скитала
def decrypt_Skitala(string, column, row):
    strings=[string[i:i+column*row] for i in range(0, len(string), column*row)]
    decryptedStr=''
    for str in strings:
        encryptMatrix = makeMatrixFromString (str, row, column)
        matrix = np.array(encryptMatrix)
        decryptMarix=matrix.transpose()
        for i in range (0, len(decryptMarix) ): 
              for j in range (0, len(decryptMarix[i]) ):
                    decryptedStr=decryptedStr+decryptMarix[i][j]
    return decryptedStr

def encode(nametext, c, n, m):
  text = readFile(nametext)
  #text = nametext   для работы без текста(просто строка)
  blocks = []
  while(len(text)!=0):
      blocks.append(text[:n*m])
      text = text[n*m:]
  #print(blocks)
  new_text =''
  for block in blocks:
    a = encrypt_Skitala(c, n, m)
    b = new_xor(a, block)
    c = b
    new_text+=b
  #print(new_text)
  writeFile(new_text, f'/content/drive/MyDrive/pyth/textpath/cript/cfb_{n}{m}.txt')
  return new_text

#функция для генерации случайного ключа
def generateKey(length):
    key = ''
    for i in range(length):
        key = key + chr(random.randint(65, 122))
    return key


def decode(nametext, c, n, m):
  text = readFile(nametext)
  #text = nametext  для работы без текста
  blocks = []
  while(len(text)!=0):
      blocks.append(text[:n*m])
      text = text[n*m:]
  #print(blocks)
  blocks_rev = list(reversed(blocks))
  block_del = blocks_rev[1]
  #print(block_del)
  new_text =''
  m1 = decrypt_Skitala(blocks_rev[1], n, m)
  #print(m1)
  for i in range(0, len(blocks_rev)-1):
     h = encrypt_Skitala(blocks_rev[i+1], n, m)
     a = new_xor(h, blocks_rev[i])
     new_text+=a
     #print(new_text)
  new_text+= new_xor(encrypt_Skitala(c, n, m), blocks_rev[len(blocks_rev)-1])
  #print(new_text)
  blockes=[]
  while(len(new_text)!=0):
      blockes.append(new_text[:n*m])
      new_text = new_text[n*m:]
  #print(blockes)
  new_blocks_rev = list(reversed(blockes))
  res = ''.join(x for x in new_blocks_rev)
  print(res)
  writeFile(res, f'/content/drive/MyDrive/pyth/textpath/cript/cfb_deco_{n}{m}.txt')
  return ''.join(x for x in new_blocks_rev)



# поиск ключа для дешифрования и расчет времени на это
def decryptGetKeys(text, encrypttext, n_m, c):
    start = time.perf_counter() 
    text = readFile(text)
    encrypttext = readFile(encrypttext)
    exp=len(text)-len(encrypttext)
    print(exp)
    numer=1;
    for i in n_m:
        print(i);
        if(len(encrypttext)%(i[0]*i[1])==0):
            print("ДААААААААААААААААААААААААААААААААААААААААААААААААААААА")
            decrypttext = decode(encrypttext, c, i[0], i[1]);
            print(encrypttext)
            print(decrypttext)
            print(text)
            if(decrypttext[:exp] == text):
                stop = time.perf_counter();
                print( "Text ", stop-start)
                print("Ключ1")
                numer+=1
                print(numer)
            else: print("no")
                


 # рассчитываем экнтропию текста
def getEntropy(fileName):
    text=readFile(fileName)
    symbols = ''.join(set(text))
    p = [text.count(symb)/len(text) for symb in symbols]
    symb_p = {symbols[i]: p[i] for i in range(0, len(p))}
    
    entropy = 0
    for ch in text:
        entropy += symb_p[ch]*math.log(1/symb_p[ch], 10)
    return entropy


# массив всевозможных пар ключей(n m) в диапазоне range(12,32):   
def getKeys():
    listed=[];
    ls=[];
    for a in range(12,32):
        for i in range(2, a // 3):
            if (a % i== 0):
                ls.append(a)
        listed= list(set(ls))
    keys_m_n =[]
    for i in range(0, len(listed)):
        primeNum = []
        for num in range(1, listed[i]+1):
            if(listed[i]%num==0):
                if(num!=1 and num!=listed[i]):
                    primeNum.append(num)
    
        com_set = itertools.combinations(primeNum, 2)
        for couple in com_set: 
            if couple[0]*couple[1]==listed[i]:
                reverse_couple=(couple[1], couple[0])
                keys_m_n.append(couple)
                keys_m_n.append(reverse_couple)
    return keys_m_n


nm=getKeys();
#print(nm)  

"""
#Для проверки кода
message = 'messagebshul'
#print(chr(0))
#key = generateKey(4)
key = '3456'
print("BIN MESSAGE: ", text_to_vector(key))
u = encode(message, key, 2, 2)
print("MESSAGE: ", u)
v = decode(u, key, 2, 2)
print(v)
"""


key = generateKey(24)
# Text1
t = readFile('/content/drive/MyDrive/pyth/textpath/cript/cfbexample.txt')
print(len(t))
print(t)
start = time.perf_counter()
t1 = encode('/content/drive/MyDrive/pyth/textpath/cript/cfbexample.txt', key, 12, 2)
print(t1)
stop = time.perf_counter()
print('Время на шифрование ' + str(stop - start) +'   Энтропия исход текста  ' + str(getEntropy('/content/drive/MyDrive/pyth/textpath/cript/cfbexample.txt'))+
      '  Энтропия зашифров текста  ' + str(getEntropy('/content/drive/MyDrive/pyth/textpath/cript/cfb_122.txt')))
start = time.perf_counter()
t2 = decode('/content/drive/MyDrive/pyth/textpath/cript/cfb_122.txt', key, 12, 2)
print(t2)
stop = time.perf_counter()
print('Время на дешифрование ' + str(stop - start))



start = time.perf_counter()
decryptGetKeys('/content/drive/MyDrive/pyth/textpath/cript/cfbexample.txt', '/content/drive/MyDrive/pyth/textpath/cript/cfb_122.txt', nm, key)
stop = time.perf_counter()
print('Перебор ключа ' + str(stop - start))


856
When he had been younger, Harry had dreamed and dreamed of some unknown relation coming to take him away, but it had never happened; the Dursleys were his only family. Yet sometimes he thought (or maybe hoped) that strangers in the street seemed to know him. Very strange strangers they were, too. A tiny man in a violet top hat had bowed to him once while out shopping with Aunt Petunia and Dudley. After asking Harry furiously if he knew the man, Aunt Petunia had rushed them out of the shop without buying anything. A wild-looking old woman dressed all in green had waved merrily at him once on a bus. A bald man in a very long purple coat had actually shaken his hand in the street the other day and then walked away without a word. The weirdest thing about all these people was the way they seemed to vanish the second Harry tried to get a closer look

)i%p <R83m 5	:$NXOKGON
EL
YEGO
_DADE]D
]G]K RL\Y
]OXO
BCY
EDFS
LKGCFS
 !~f.<"o,%".yg*&c21<c-~^XKDMOXY
CD
^BO
Y^XOO^
Y;;5'/o0em2

ValueError: ignored